<a href="https://colab.research.google.com/github/christoph-fraller/dopp_2020w_group03_ex3/blob/main/dopp_2020w_group03_ex3_with_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate SSH-Keys for Accessing Git Repository

In [ ]:
# import and mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# generate ssh keys (insert your username@github.com + hit enter when prompted for any answer)
! ssh-keygen -t rsa -b 4096 -C 'yummakan@github.com'

In [ ]:
# check whether or not the ssh keys have been created ('id_rsa' and 'id_rsa.pub' should be displayed)
! ls /root/.ssh/

In [ ]:
# create directory for saving the ssh keys
! mkdir -p /content/drive/MyDrive/Ssh

In [ ]:
# copy ssh keys from /root/.ssh/* to /content/drive/MyDrive/Ssh/*
! cp /root/.ssh/id_rsa /content/drive/MyDrive/Ssh/
! cp /root/.ssh/id_rsa.pub /content/drive/MyDrive/Ssh/

In [ ]:
# display public ssh key for copy/paste
! cat /content/drive/MyDrive/Ssh/id_rsa.pub

In [ ]:
# add github to known hosts and adapt file access permissions
! ssh-keyscan github.com >> /root/.ssh/known_hosts
! chmod 644 /root/.ssh/known_hosts
! chmod 600 /root/.ssh/id_rsa
! ssh -T git@github.com

# Git Setup

In [ ]:
# git config settings (replace with your credentials)
! git config --global user.email "maximilian.loesch97@gmail.com"
! git config --global user.name "Maxiking1997"

In [ ]:
# create directory for git repositories
! mkdir -p /content/drive/MyDrive/Git

In [ ]:
# git-clone has to be performed only once when setting up the git repo at your google drive
! git clone git@github.com:christoph-fraller/dopp_2020w_group03_ex3.git /content/drive/MyDrive/Git/dopp_2020w_group03_ex3

## Important Shell and Git Commands


**NOTICE:** Always ensure that you are in the right directory when performing git commands (e.g. /content/drive/MyDrive/Git/dopp_2020w_group03_ex3). In case of any issues that might occur when switching directories it is highly recommended to restart the runtime engine (CTRL + M + .).

In [ ]:
# check current working directory
! pwd

In [ ]:
# switch to specified working directory
%cd /content/drive/MyDrive/Git/dopp_2020w_group03_ex3

In [ ]:
# list content of current working directory
! ls

In [ ]:
# check git status
! git status

In [ ]:
# always perform a git pull before you start working or commit/push some changes
! git pull

In [ ]:
# add a new data file to git repo directly from colab
# at first upload the file into the folder of your google drive
#! git add data/data_indicators_2.csv
#! git commit -m 'New file added.'
#! git push

# Perform these steps everytime when a new session has been started

In [ ]:
# import and mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# create directory
! mkdir -p /root/.ssh

In [ ]:
# copy ssh keys from /content/drive/MyDrive/Ssh/* to /root/.ssh/*
! cp /content/drive/MyDrive/Ssh/id_rsa /root/.ssh/
! cp /content/drive/MyDrive/Ssh/id_rsa.pub /root/.ssh/ 

In [ ]:
# add github to known hosts and adapt file access permissions
! ssh-keyscan github.com >> /root/.ssh/known_hosts
! chmod 644 /root/.ssh/known_hosts
! chmod 600 /root/.ssh/id_rsa
! ssh -T git@github.com

In [ ]:
# switch to specified working directory
%cd /content/drive/MyDrive/Git/dopp_2020w_group03_ex3

In [ ]:
# always perform a git pull before you start working or commit/push some changes
! git pull

## Geopanda installation

In [ ]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sb
import geopandas
from ipywidgets import IntSlider, interact
from scipy import stats
from statistics import mean

# Data Preprocessing

## Load and merge income data


In [ ]:
def load_merge_income_data():
  
  # load income data from csv
  income_data = pd.read_csv('/content/drive/MyDrive/Git/dopp_2020w_group03_ex3/data/undata_gni_per_capita.csv', sep = ';')

  # extend income data by adding an entry for each combination of (calendar_year, country_code) due to there are currently no missing entries in the data
  country_data_list = income_data[['country_code', 'country_name']].drop_duplicates().values.tolist()
  output_list = []
  for lst in country_data_list:
      country_code = lst[0]
      country_name = lst[1]
      for calendar_year in range(1970, 2019):
        output_list.append([calendar_year, country_code, country_name])
  df = pd.DataFrame(output_list, columns = ['calendar_year', 'country_code', 'country_name'])
  income_data_complete = df.merge(income_data[['calendar_year', 'country_code', 'gni_per_capita_us_dollar']], how = 'left', on = ['calendar_year', 'country_code'])

  # load population data from csv
  population_data = pd.read_csv('/content/drive/MyDrive/Git/dopp_2020w_group03_ex3/data/undata_population_total.csv', sep = ';', 
                                usecols = ['country_code', 'calendar_year', 'population_total'])
  population_data.drop_duplicates(inplace = True)
  population_data['population_total'] = population_data['population_total'] * 1000 # total population is specified in 1000

  # load country codes from csv
  country_codes = pd.read_csv('/content/drive/MyDrive/Git/dopp_2020w_group03_ex3/data/iso3166_unsd_country_codes.csv', sep = ';', 
                              usecols = ['m49_code', 'iso_alpha2_code', 'iso_alpha3_code', 'small_island_developing_states'])

  # merge income with population data and country codes
  output_data = income_data_complete.merge(population_data, how = 'left', on = ['calendar_year', 'country_code'])
  output_data = output_data.merge(country_codes, how = 'left', left_on = 'country_code', right_on = 'm49_code')

  return output_data

merged_income_data = load_merge_income_data()

## Clean issues in income data

In [ ]:
def clean_income_data(input_data):
  
  output_data = input_data.copy()

  # drop duplicate information at column-level
  output_data.drop(['country_code'], axis = 1, inplace = True)
  output_data.rename(columns = {'country_code': 'm49_code'}, inplace = True) 

  # replace values of column 'small_island_developing_states' by True/False
  output_data['small_island_developing_states'].replace('x', True, inplace = True)
  output_data['small_island_developing_states'].fillna(False, inplace = True)

  # fix issues at column 'small_island_developing_states' for some countries
  output_data.iloc[output_data[output_data['country_name'] == 'Former Netherlands Antilles'].index, output_data.columns.get_loc('small_island_developing_states')] = True
  output_data.iloc[output_data[output_data['country_name'] == 'United Republic of Tanzania: Mainland'].index, output_data.columns.get_loc('small_island_developing_states')] = True
  output_data.iloc[output_data[output_data['country_name'] == 'United Republic of Tanzania: Zanzibar'].index, output_data.columns.get_loc('small_island_developing_states')] = True

  # drop small island countries and reset row index
  output_data.drop(output_data[output_data['small_island_developing_states'] == True].index, inplace = True)
  output_data.reset_index(drop = True, inplace = True)

  # reorder colums of dataframe
  output_data = output_data[['calendar_year', 'iso_alpha3_code', 'iso_alpha2_code', 'm49_code', 'country_name', 'population_total', 'gni_per_capita_us_dollar']]

  return output_data

cleaned_income_data = clean_income_data(merged_income_data)

## Exploring missing values in income data

In [ ]:
# get an overview of missing values in income data
# it seems there is pattern between the missing values of the columns iso_alpha3_code, iso_alpha2_code, m49_code and population_total
ax = plt.axes()
sb.heatmap(cleaned_income_data.isna(), cbar = False);
ax.set_title('Visualization of missing values in income dataset')
plt.show()

In [ ]:
# ad 1) missing values in country codes
# obtain countries with missing country codes (iso_alpha3_code, iso_alpha2_code, m49_code)
iso_alpha3_code_set = set(cleaned_income_data.loc[cleaned_income_data['iso_alpha3_code'].isna()].country_name.unique().tolist())
iso_alpha2_code_set = set(cleaned_income_data.loc[cleaned_income_data['iso_alpha2_code'].isna()].country_name.unique().tolist())
m49_code_set = set(cleaned_income_data.loc[cleaned_income_data['m49_code'].isna()].country_name.unique().tolist())
union_list_sorted = sorted(iso_alpha3_code_set | iso_alpha2_code_set | m49_code_set)
print('\nCountries with missing entries in their country codes:')
print('------------------------------------------------------')
print(*union_list_sorted, sep = '\n')

# strategy on dealing with missing in country codes:

## most of the missing entries in country codes can be traced back to former countries that no longer exist and therefore their codes are missing in 
## the actual iso3166 standard but in order to obtain a complete dataset we will refill based on historical data: Former Czechoslovakia, Former Ethiopia,
## Former Sudan, Former USSR, Former Yugoslavia, Yemen: Former Democratic Yemen, Yemen: Former Yemen Arab Republic

## Nambia's iso_alpha2_code corresponds to 'NA', which is interpreted as NA per default, we have to fix this when inserting the other missing country codes

## Kosovo has declared its independence from Serbia in 2008 but until today this declaration is quite controversial
## due to reasons of simplicity and without being politically, we have decided to exclude the Kosovo from our analysis

In [ ]:
# ad 2) missing values in population
# obtain countries with population total 
population_list_sorted = sorted(cleaned_income_data.loc[cleaned_income_data['population_total'].isna()].country_name.unique().tolist())
print('\nCountries with missing entries in their population values:')
print('----------------------------------------------------------')
print(*population_list_sorted, sep = '\n')

# strategy on dealing with missing in population values:

## missing entries in population values can be traced back to former countries that no longer exist
## because we know the former composition of that countries we can easily calculate their population values based on their components
## at least this is a possible approach for large countries that have been splitted up: 
### Former Sudan, Former USSR, Former Yugoslavia, Yemen: Former Democratic Yemen, Yemen: Former Yemen Arab Republic
### Former Czechoslovakia -> Czech Republic, Slovakia,
### Former Ethiopia -> Ethiopia, Eritrea,
### Former Sudan -> Sudan, South Sudan],
### Former USSR -> Armenia, Azerbaijan, Belarus, Estonia, Georgia, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Republic of Moldova, Russian Federation, Tajikistan, Turkmenistan, Ukraine, Uzbekistan
### Former Yugoslavia -> Bosnia and Herzegovina, Croatia, Montenegro, Republic of North Macedonia, Serbia, Slovenia

## in case of Yemen we have a merge of two former countries for which the above mentioned approach is not possible
## even if such a merge of two quite similiar countries (at least in gni_per_capita) is politically important, for our purposes it is not
## therefore we decided to consider Yemen in our data as one country for the entire observation period

## Kosovo has declared its independence from Serbia in 2008 but until today this declaration is quite controversial
## due to reasons of simplicity and without being politically, we have decided to exclude the Kosovo from our analysis

In [ ]:
# ad 3) missing values in gni per capita
# obtain countries with gni per capita
gni_null_values = cleaned_income_data.gni_per_capita_us_dollar.isnull().groupby(cleaned_income_data['country_name']).sum().astype(int).reset_index(name = 'null_count')
print('\nCountries with missing entries in their gni per capita values:')
print('--------------------------------------------------------------')
print(gni_null_values[gni_null_values['null_count'] > 0])

# strategy on dealing with missing in gni per capita values:
## when taking a closer look at the data, almost all of the missing values at gni per capita can be traced back to years for which a country does not exist
## therefore such entries with missing values at gni per capita can be safely removed but for Yemen a special handling will be required: 
## due to the similarity of the Yemen: Former Democratic Yemen and Yemen: Former Yemen Arab Republic we decided to calculate the mean of the gni per capita
## of both countries and use it to replace the historical missing values of Yemen, for our work the separation between that two former countries is neglible

## Handling missing country codes

In [ ]:
def handle_missing_country_codes(input_data):

  output_data = input_data.copy()

  # drop entries of Kosovo
  output_data.drop(output_data[output_data['country_name'] == 'Kosovo'].index, inplace = True)
  output_data.reset_index(drop = True, inplace = True)

  # load formerly used country codes from csv, set na_filter to false in order to fix the issue of Namibia's iso_alpha2_code
  former_country_codes = pd.read_csv('/content/drive/MyDrive/Git/dopp_2020w_group03_ex3/data/iso3166_formerly_used_country_codes.csv', sep = ';', na_filter = False)

  country_codes_list = ['iso_alpha3_code', 'iso_alpha2_code', 'm49_code']   

  # insert missing values from former_country_codes
  for row_index in range(0, len(output_data)):
      
    for country_code in country_codes_list:
        
      # check if country code value is missing
      if pd.isnull(output_data.loc[row_index, country_code]):

        # insert missing country code
        output_data.loc[row_index, country_code] = former_country_codes.loc[former_country_codes[former_country_codes['country_name'] == output_data.loc[row_index, 'country_name']].index[0], country_code]
  
  return output_data
    
income_data_country_codes_complete = handle_missing_country_codes(cleaned_income_data)

## Compute missing population data

In [ ]:
def compute_missing_population_total(input_data):

    output_data = input_data.copy()

    former_country_lists = [['Former Czechoslovakia', 'Czech Republic', 'Slovakia'],
                             ['Former Ethiopia', 'Ethiopia', 'Eritrea'],
                             ['Former Sudan', 'Sudan', 'South Sudan'],
                             ['Former USSR', 'Armenia', 'Azerbaijan', 'Belarus', 'Estonia', 'Georgia', 'Kazakhstan', 'Kyrgyzstan', 'Latvia', 'Lithuania', 
                              'Republic of Moldova', 'Russian Federation', 'Tajikistan', 'Turkmenistan', 'Ukraine', 'Uzbekistan'],
                             ['Former Yugoslavia', 'Bosnia and Herzegovina', 'Croatia', 'Montenegro', 'Republic of North Macedonia', 'Serbia', 'Slovenia']]

    for country_list in former_country_lists:

      # get first list elem of list
      former_country_name = country_list.pop(0)

      # get list of calendar_years
      calendar_year_list = output_data.loc[output_data['country_name'] == former_country_name].calendar_year.tolist()

      for calendar_year in calendar_year_list:
        
        # get current index of former_country at particular calender_year
        curr_former_country_index = output_data.loc[(output_data['calendar_year'] == calendar_year) & (output_data['country_name'] == former_country_name)].population_total.index.max()
        
        # initialize new_population_total
        new_population_total = 0

        for country_name in country_list:
          
          if pd.isnull(output_data.loc[(output_data['calendar_year'] == calendar_year) & (output_data['country_name'] == country_name)].gni_per_capita_us_dollar).bool():
            
            # get current index of country at particular calender_year  
            curr_country_index = output_data.loc[(output_data['calendar_year'] == calendar_year) & (output_data['country_name'] == country_name)].population_total.index.max()

            # increment new_population_total by population_total of curr_country_index at particular calender_year 
            new_population_total += output_data.loc[curr_country_index, 'population_total']

        # update population_total of former_country with new_population_total at particular calender_year 
        output_data.loc[curr_former_country_index, 'population_total'] = new_population_total if new_population_total > 0 else np.nan

    return output_data
    
income_data_population_total_complete = compute_missing_population_total(income_data_country_codes_complete)

## Fix missing gni per capita values

In [ ]:
def fix_missing_gni_per_capita_values(input_data):
  
  output_data = input_data.copy()
  
  # set index on calendar_year
  output_data.set_index(['calendar_year'], inplace = True)

  # special handling for 'Yemen': 
  # compute mean of gni per capita from 'Yemen: Former Democratic Yemen' and 'Yemen: Former Yemen Arab Republic'
  # and replace missing values of 'Yemen' with the computed mean
  yemen_missing_gni_values = (output_data[output_data['country_name'] == 'Yemen: Former Democratic Yemen'].gni_per_capita_us_dollar + output_data[output_data['country_name'] == 'Yemen: Former Yemen Arab Republic'].gni_per_capita_us_dollar) / 2
  yemen_missing_gni_values.dropna(inplace = True)
  yemen_df = pd.DataFrame(output_data[output_data['country_name'] == 'Yemen'].gni_per_capita_us_dollar.fillna(yemen_missing_gni_values))
  yemen_df['country_name'] = 'Yemen'
  yemen_df.reset_index(inplace = True)
  yemen_df.set_index(['calendar_year', 'country_name'], inplace = True)
  
  # fill missing values of 'Yemen' with above computed values
  output_data.reset_index(inplace = True)
  output_data.set_index(['calendar_year', 'country_name'], inplace = True)
  output_data.fillna(yemen_df, inplace = True)

  # reset index
  output_data.reset_index(inplace = True)
  
  # drop 'Yemen: Former Democratic Yemen' and 'Yemen: Former Yemen Arab Republic' due to consolidation
  output_data.drop(output_data[output_data['country_name'] == 'Yemen: Former Democratic Yemen'].index, inplace = True)
  output_data.drop(output_data[output_data['country_name'] == 'Yemen: Former Yemen Arab Republic'].index, inplace = True)
  
  # drop entries that still involve missing values is now safe 
  output_data.dropna(inplace = True)
  output_data.reset_index(inplace = True, drop = True)

  return output_data

income_data_preprocessed = fix_missing_gni_per_capita_values(income_data_population_total_complete)

# check whether or not all missing values have been replaced
sb.heatmap(income_data_preprocessed.isna(), cbar = False);

## Load thresholds and sdr deflators


In [ ]:
def load_thresholds_sdr_deflators():
  
  # load income-level thresholds
  income_threshold_data = pd.read_csv('/content/drive/MyDrive/Git/dopp_2020w_group03_ex3/data/world_bank_income_level_thresholds.csv', sep = ',')
  income_threshold_data.drop(['banks_fiscal_year'], axis = 1, inplace = True)

  # load sdr deflator values
  sdr_deflator_data = pd.read_csv('/content/drive/MyDrive/Git/dopp_2020w_group03_ex3/data/world_bank_sdr_deflator.csv', sep = ',')
  sdr_deflator_data.drop(['sdr_deflator_us_dollar'], axis = 1, inplace = True)

  # create index on calendar_year
  income_threshold_data.set_index(['calendar_year'], inplace = True)
  sdr_deflator_data.set_index(['calendar_year'], inplace = True)

  # merge income_threshold_data and sdr_deflator_data using index 'calendar_year'
  output_data = income_threshold_data.merge(sdr_deflator_data, how = 'inner', left_index = True, right_index = True)

  return output_data

threshold_data = load_thresholds_sdr_deflators()

# visualize missing threshold values using a heatmap
sb.heatmap(threshold_data[['low_income_level_threshold', 'middle_income_level_threshold', 'high_income_level_threshold']].isna(), cbar = False);

## Calculate missing thresholds using sdr deflators


In [ ]:
def calculate_missing_thresholds(input_data):
    
    output_data = input_data.copy()
    
    thresholds_list = ['low_income_level_threshold', 'middle_income_level_threshold', 'high_income_level_threshold']   
    
    # calculate missing values
    for row_index in range(0, len(output_data)):
      
      # retrieve current index
      curr_index = output_data.index.max() - row_index
      for threshold in thresholds_list:
        
        # check if threshold value is missing
        if(np.isnan(output_data.loc[curr_index, threshold])):
          
          # calculate missing threshold value based on existing threshold and sdr_inflation_rate_annual_change
          output_data.loc[curr_index, threshold] = output_data.loc[curr_index + 1, threshold] / (100 + output_data.loc[curr_index + 1, 'sdr_inflation_rate_annual_change']) * 100
    
    # round thresholds to nearest multiple of base
    base = 5
    for threshold in thresholds_list:
      output_data[threshold] = round(output_data[threshold] / base) * base

    return output_data[thresholds_list]

threshold_data_preprocessed = calculate_missing_thresholds(threshold_data)

# check whether or not all missing values have been replaced
sb.heatmap(threshold_data_preprocessed.isna(), cbar = False);

## Assign income-level labels based on gni and income-level thresholds

In [ ]:
def assign_income_level_labels(input_income_data, input_threshold_data):

  # merge income_data_preprocessed and threshold_data_complete using calendar_year
  output_data = input_income_data.merge(input_threshold_data, how = 'left', on = 'calendar_year')

  # assign income-level labels based on gni and thresholds
  output_data.loc[(output_data['gni_per_capita_us_dollar'] <= output_data['low_income_level_threshold']), 'income_level_label'] = 'low_income'
  output_data.loc[((output_data['gni_per_capita_us_dollar'] > output_data['low_income_level_threshold']) & 
                          (output_data['gni_per_capita_us_dollar'] <= output_data['middle_income_level_threshold'])), 'income_level_label'] = 'lower_middle_income'
  output_data.loc[((output_data['gni_per_capita_us_dollar'] > output_data['middle_income_level_threshold']) & 
                          (output_data['gni_per_capita_us_dollar'] <= output_data['high_income_level_threshold'])), 'income_level_label'] = 'upper_middle_income'
  output_data.loc[(output_data['gni_per_capita_us_dollar'] > output_data['high_income_level_threshold']), 'income_level_label'] = 'high_income'

  # create index on calendar_year
  output_data.set_index(['calendar_year'], inplace = True)

  return output_data

income_classification_data = assign_income_level_labels(income_data_preprocessed, threshold_data_preprocessed)

# Question 1

## Calculating the population living in the different income levels

In [ ]:
def calculate_sum_population_per_income_level_and_year(input_data):

  output_data = pd.DataFrame()

  # create sum of population of each income category for each year
  output_data['low_income_population'] = input_data['population_total'].loc[input_data['income_level_label'] == 'low_income'].groupby('calendar_year').sum()
  output_data['lower_middle_income_population'] = input_data['population_total'].loc[input_data['income_level_label'] == 'lower_middle_income'].groupby('calendar_year').sum()
  output_data['upper_middle_income_population'] = input_data['population_total'].loc[input_data['income_level_label'] == 'upper_middle_income'].groupby('calendar_year').sum()
  output_data['high_income_population'] = input_data['population_total'].loc[input_data['income_level_label'] == 'high_income'].groupby('calendar_year').sum()
  output_data['total_population'] = input_data['population_total'].groupby('calendar_year').sum()

  return output_data

population_income = calculate_sum_population_per_income_level_and_year(income_classification_data)

## Visualization of results

In [ ]:
# plot cumulative total population according to income-levels

# select years to include in plot
years = range(min(population_income.index), max(population_income.index))

# calculate cumulative total_population according to income-levels
low_income_cumulative = population_income.loc[population_income.index.isin(years), 'low_income_population'].values
lower_middle_income_cumulative = low_income_cumulative + population_income.loc[population_income.index.isin(years), 'lower_middle_income_population'].values
upper_middle_income_cumulative = lower_middle_income_cumulative + population_income.loc[population_income.index.isin(years), 'upper_middle_income_population'].values
high_income_cumulative = upper_middle_income_cumulative + population_income.loc[population_income.index.isin(years), 'high_income_population'].values

# create plot
fig, ax = plt.subplots()

ax.plot(population_income['total_population'].groupby('calendar_year').max(), label = 'total_population', color = 'Black')
ax.plot(population_income['total_population'].groupby('calendar_year').max() * 0.5, label = 'half_of_total_population', color = 'Red', linestyle = 'dashed')

ax.fill_between(years ,upper_middle_income_cumulative, high_income_cumulative, alpha = 0.3)
ax.fill_between(years ,lower_middle_income_cumulative, upper_middle_income_cumulative, alpha= 0.3)
ax.fill_between(years, low_income_cumulative, lower_middle_income_cumulative, alpha = 0.3)
ax.fill_between(years, low_income_cumulative, 0, alpha = 0.3)
ax.text(1990, 1.5e9, 'low income', fontsize = 9, color = 'White')
ax.text(1995, 3.5e9, 'lower_middle_income', fontsize = 9, color = 'White')
ax.text(2003, 5e9, 'upper_middle_income', fontsize = 9, color = 'White')
ax.text(2009, 6.5e9, 'high_income', fontsize = 9, color = 'White')

ax.set_title('Visualizing historical development of world\'s population according to different \n income-levels based on the information of more than 160 countries', pad = 20)
ax.set_xlabel('Calendar Year')
ax.set_ylabel('World\'s Population')

plt.xlim([min(population_income.index), max(population_income.index) - 1])
plt.ylim([0, population_income['total_population'].max()])
plt.legend()

plt.show()

In [ ]:
# plot income of the 5 largest countries 

# select years to include in plot
years = range(min(population_income.index), max(population_income.index))

# select 5 largest countries
countries_most_populous = income_classification_data.loc[max(population_income.index)].nlargest(5,'population_total')['country_name'].values

# extract income thresholds
high_income_treshold = threshold_data_preprocessed[threshold_data_preprocessed.index.isin(years)]['high_income_level_threshold'].values
middle_income_treshold = threshold_data_preprocessed[threshold_data_preprocessed.index.isin(years)]['middle_income_level_threshold'].values
low_income_treshold = threshold_data_preprocessed[threshold_data_preprocessed.index.isin(years)]['low_income_level_threshold'].values

# create plot
fig, ax = plt.subplots()

for i in range(5):
  ax.plot(income_classification_data['gni_per_capita_us_dollar'].loc[income_classification_data['country_name'] == countries_most_populous[i]],label = countries_most_populous[i])

plt.yscale('log')

ax.fill_between(years,high_income_treshold, 1e5,alpha = 0.3)
ax.fill_between(years,high_income_treshold, middle_income_treshold, alpha = 0.3)
ax.fill_between(years,low_income_treshold, middle_income_treshold, alpha = 0.3)
ax.fill_between(years,low_income_treshold, 0, alpha = 0.3)
ax.text(2004, 1.5e2, 'low income', fontsize = 9, color = 'White')
ax.text(1980, 1.3e3, 'lower_middle_income', fontsize = 9, color = 'White')
ax.text(1990, 5e3, 'upper_middle_income', fontsize = 9, color = 'White')
ax.text(2000, 2e4, 'high_income', fontsize = 9, color = 'White')

ax.set_title('TODO', pad = 20)
ax.set_xlabel('Calendar Year')
ax.set_ylabel('log(GNI per Capita in US Dollar)')

plt.xlim([min(population_income.index), max(population_income.index) - 1])
plt.ylim([5e1,1e5])
plt.legend()

plt.show()

In [ ]:
# show histogram of differtent income levels in 2018
fig, ax = plt.subplots()
income_classification_data['income_level_label'].loc[max(income_classification_data.index)].hist( grid = False)
ax.set_title('Distribution of countries according to different income-levels', pad = 20)
plt.xlabel('Income Levels')
plt.ylabel('Number of Countries')

plt.show() 

In [ ]:
# create df to show the sum of coutries for each income classification
income_classification_count = income_classification_data['income_level_label'].groupby('calendar_year').value_counts(dropna = False)

# plot total count of classification entries each year [sum of countries]
fig, ax = plt.subplots()
income_classification_total_count = income_classification_count[:,'high_income']+income_classification_count[:,'upper_middle_income']+income_classification_count[:,'lower_middle_income']+income_classification_count[:,'low_income']
plt.plot(income_classification_total_count, label = 'total_classification_count', color = 'black')
ax.set_title('TODO', pad = 20)
plt.xlabel('Calendar Year')
plt.ylabel('Number of Countries')
plt.xlim([min(income_classification_total_count.index), max(income_classification_total_count.index) - 1])
plt.ylim([0, 200])
plt.legend()
plt.show()

In [ ]:
# wollt ihr die beiden plots drinnen lassen, glaub die sind recht schwer zu interpretieren? 
# plot income classification count over time
plt.plot(population_income['high_income_population'], label = 'high_income' )
plt.plot(population_income['upper_middle_income_population'], label = 'upper_middle_income')
plt.plot(population_income['lower_middle_income_population'], label = 'lower_middle_income')
plt.plot(population_income['low_income_population'], label = 'low_income')
plt.xlabel('Calendar Year')
plt.ylabel('Number of People')
plt.legend()
plt.show()

# income_classification_data =  income_classification_data.loc[income_classification_data['population_total'] > 1000000.0]
# print(income_classification_data)
# print(income_classification_data[['population_total', 'income_level_label']].groupby(['calendar_year', 'income_level_label']).sum().groupby('calendar_year').sum())

# plot income classification count over time [sum of countries]
plt.plot(income_classification_count[:,'high_income'], label = 'high_income' )
plt.plot(income_classification_count[:,'upper_middle_income'], label = 'upper_middle_income')
plt.plot(income_classification_count[:,'lower_middle_income'], label = 'lower_middle_income')
plt.plot(income_classification_count[:,'low_income'], label = 'low_income')
plt.xlabel('Calendar Year')
plt.ylabel('Number of Countries')
plt.legend()
plt.show()

## Interactive world map (choropleth map)

In [ ]:
def preprocessing_world_map_data(input_data):

  output_data = input_data.copy()

  # drop index on calendar_year
  output_data.reset_index(inplace = True)

  former_country_lists = [['Former Czechoslovakia', 'Czech Republic', 'Slovakia'],
                             ['Former Ethiopia', 'Ethiopia', 'Eritrea'],
                             ['Former Sudan', 'Sudan', 'South Sudan'],
                             ['Former USSR', 'Armenia', 'Azerbaijan', 'Belarus', 'Estonia', 'Georgia', 'Kazakhstan', 'Kyrgyzstan', 'Latvia', 'Lithuania', 
                              'Republic of Moldova', 'Russian Federation', 'Tajikistan', 'Turkmenistan', 'Ukraine', 'Uzbekistan'],
                             ['Former Yugoslavia', 'Bosnia and Herzegovina', 'Croatia', 'Montenegro', 'Republic of North Macedonia', 'Serbia', 'Slovenia']]

  for country_list in former_country_lists:

    # get first list elem of list
    former_country_name = country_list.pop(0)

    # get list of calendar_years
    calendar_year_list = output_data.loc[output_data['country_name'] == former_country_name].calendar_year.tolist()
    
    for calendar_year in calendar_year_list:

      # get current index of former country at particular calender_year  
      curr_former_country_index = output_data.loc[(output_data['calendar_year'] == calendar_year) & (output_data['country_name'] == former_country_name)].income_level_label.index.max()

      # get income-level of former country
      income_level_label = output_data.loc[curr_former_country_index, 'income_level_label']
      
      # get max calender year of dataframe
      max_calendar_year = max(output_data['calendar_year'])

      for country_name in country_list:
        
        # get current index of country at particular calender_year  
        curr_country_index = output_data.loc[(output_data['calendar_year'] ==  max_calendar_year) & (output_data['country_name'] == country_name)].income_level_label.index.max()

        # get income-level of former country
        iso_alpha3_code = output_data.loc[curr_country_index, 'iso_alpha3_code']

        # append row to dataframe
        output_data = output_data.append({'calendar_year' : calendar_year, 
                                          'iso_alpha3_code' : iso_alpha3_code,
                                          'country_name' :  country_name, 
                                          'income_level_label' : income_level_label} , ignore_index = True)

  # set index on calendar_year
  output_data.reset_index(inplace = True, drop = True)
  output_data.sort_values(by = ['country_name', 'calendar_year'], inplace = True)
  output_data.set_index('calendar_year', inplace = True)

  return output_data

world_map_data = preprocessing_world_map_data(income_classification_data)

In [ ]:
@interact(selected_year = IntSlider(value = min(world_map_data.index), min = min(world_map_data.index), max = max(world_map_data.index)))
def interactive_world_map_visualization_(selected_year):
  
  # load geometry data for all countries 
  world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

  # correct wrong iso code
  world.loc[world['name'] == 'France', 'iso_a3'] = 'FRA'
  world.loc[world['name'] == 'Norway', 'iso_a3'] = 'NOR'
  world.loc[world['name'] == 'Kosovo', 'iso_a3'] = 'XKX'

  # drop unnecessary columns
  country_shapes = world[['geometry', 'iso_a3']]

  # merge geometry with income data on country code, select year to look at
  geo_income_data = country_shapes.merge(world_map_data.loc[selected_year], left_on='iso_a3', right_on='iso_alpha3_code')

  # plot map
  legend_dict = {'bbox_to_anchor' : (1., 1), 'loc' :'upper left'}
  colors = ['dodgerblue','maroon','lightcoral','skyblue']
  geo_income_data.plot(column='income_level_label', legend = True, legend_kwds = legend_dict, figsize=(15, 10), cmap = matplotlib.colors.ListedColormap(colors))

# Question 2

## Data Preprocessing

### Load Data

In [ ]:
def load_world_bank_indicator_data():
  # load indicator data from csv (World Bank Data)
  data = pd.read_csv('/content/drive/MyDrive/Git/dopp_2020w_group03_ex3/data/data_indicators_2.csv', sep = ',', nrows = 16104)
  data.replace('..',np.nan,inplace=True)

  #rename columns
  rename_columns_dict = {'Time':'calendar_year','Country Code':'iso_alpha3_code','Fertility rate, total (births per woman) [SP.DYN.TFRT.IN]':'fertility_rate','Urban population (% of total population) [SP.URB.TOTL.IN.ZS]':'urban_population','Access to electricity (% of population) [EG.ELC.ACCS.ZS]':'access_electricity', 'Agriculture, forestry, and fishing, value added (% of GDP) [NV.AGR.TOTL.ZS]':'agriculture_forestry_fishing_sector','Unemployment, total (% of total labor force) (modeled ILO estimate) [SL.UEM.TOTL.ZS]':'unemployment', 'Total natural resources rents (% of GDP) [NY.GDP.TOTL.RT.ZS]':'natural_resources_rent','Inflation, consumer prices (annual %) [FP.CPI.TOTL.ZG]':'inflation','External balance on goods and services (% of GDP) [NE.RSB.GNFS.ZS]':'external_balance_on_goods_and_services','School enrollment, primary (% gross) [SE.PRM.ENRR]':'primary_school_enrollment','Life expectancy at birth, total (years) [SP.DYN.LE00.IN]':'life_expectancy'}
  data.rename(rename_columns_dict, axis='columns',inplace=True)

  #only use Data from 2013-2018
  data = data[data['calendar_year'].isin(range(2013,2019))]

  # select only ceratain parameters (drop columns with very few entries)
  data = data[['calendar_year','iso_alpha3_code','fertility_rate','urban_population','access_electricity', 'agriculture_forestry_fishing_sector','external_balance_on_goods_and_services','natural_resources_rent','inflation','life_expectancy']]

  #change data types of columns
  data = data.astype({'fertility_rate': float, 'urban_population': float, 'access_electricity': float, 'agriculture_forestry_fishing_sector': float, 'inflation': float, 'external_balance_on_goods_and_services': float, 'natural_resources_rent': float, 'life_expectancy': float})

  return data

indicator_data = load_world_bank_indicator_data()

### Merge with Data from Question 1

In [ ]:
def merge_indicator_and_income_data(indicator, income):
  income.reset_index(inplace =True)
  merged_data = income.merge(indicator, how = 'inner', on = ['calendar_year','iso_alpha3_code'])

  #set index 
  merged_data.set_index(['calendar_year','country_name'], inplace = True)

  return merged_data

indicator_income_data = merge_indicator_and_income_data(indicator_data, income_classification_data)

### Deal with missing data and outliers

In [ ]:
sb.heatmap(indicator_income_data.isna(), cbar = False);

In [ ]:
def clean_data(input_data):

  #delete data entries with inflation over 30 % . Ignore hyperinflation in Sudan, Venezuela, etc...
  output_data= input_data.loc[indicator_income_data['inflation']<25]

  #remove specified countries that have very few data available
  drop_countries_list = ['AND', 'CYM','MCO', 'SMR', 'TCA', 'ERI','PRK','SOM']
  output_data = output_data[output_data['iso_alpha3_code'].isin(drop_countries_list) == False]

  # fill the missing agriculture_forestry_fishing_sector values in 2017 and 2018 in Canada with the value from the year 2016. 
  output_data.loc[([2017,2018],'Canada'),'agriculture_forestry_fishing_sector'] = 1.862226	

  # fill the missing agriculture_forestry_fishing_sector values in  2018 in New Zealand with the value from the year 2017. 
  output_data.loc[([2018],'New Zealand'),'agriculture_forestry_fishing_sector'] = 5.814862	

  # The parameter agriculture_forestry_fishing_sector does not change rapidly and therfore filling it with the last known value should be justified for Canada and New Zealand. Only 1 or 2 years missing

  # fill the missing agriculture_forestry_fishing_sector values for Macau with zero. 
  output_data.loc[([2013,2014,2015,2016,2017,2018],'China, Macao Special Administrative Region'),'agriculture_forestry_fishing_sector'] = 0	

  #no agriculture_forestry_fishing_sector data was available in the data of the world bank. Macau is a very small urban area with nearly no agriculture_forestry_fishing_sector.  https://www.indexmundi.com/macau/economy_profile.html
  #Remove all countries with missing external_balance_on_goods_and_services or agriculture_forestry_fishing_sector data
  output_data.dropna(subset = ['fertility_rate','urban_population','access_electricity', 'agriculture_forestry_fishing_sector','external_balance_on_goods_and_services','natural_resources_rent','inflation','life_expectancy'],inplace = True)

  return output_data


indicator_income_data_cleaned = clean_data(indicator_income_data)

In [ ]:
indicator_income_data_cleaned

### Deal with outlieres

In [ ]:
#indicator_income_data[(indicator_income_data['urban_population']<40) & (indicator_income_data['gni_per_capita_us_dollar']>1e4)]

#Liechtenstein is a big outlier regarding the low urban population and the big gni and also a very small country (Was clearly visible in scatter plots). Therfore it is droped for our algorithm ?????
#indicator_income_data.drop('Liechtenstein', level='country_name', inplace=True)

# Uni-variate outlier analysis
# Discovering outliers with visualization tool Box plot

fig, axs = plt.subplots(2,4)
axs[0,0].boxplot(indicator_income_data_cleaned['fertility_rate'])
axs[0,0].set_title('fertility_rate')
axs [0,1].boxplot(indicator_income_data_cleaned['urban_population'])
axs [0,1].set_title('urban_population')
axs[0,2].boxplot(indicator_income_data_cleaned['access_electricity'])
axs[0,2].set_title('access_electricity')
axs [0,3].boxplot(indicator_income_data_cleaned['agriculture_forestry_fishing_sector'])
axs [0,3].set_title('agriculture_forestry_fishing_sector')
axs[1,0].boxplot(indicator_income_data_cleaned['external_balance_on_goods_and_services'])
axs[1,0].set_title('external_balance_on_goods_and_services')
axs [1,1].boxplot(indicator_income_data_cleaned['natural_resources_rent'])
axs [1,1].set_title('natural_resources_rent')
axs[1,2].boxplot(indicator_income_data_cleaned['inflation'])
axs[1,2].set_title('inflation')
axs [1,3].boxplot(indicator_income_data_cleaned['life_expectancy'])
axs [1,3].set_title('life_expectancy1')

fig.subplots_adjust(left=0.08, right=0.98, bottom=0.05, top=0.9,
                    hspace=0.6, wspace=0.6)


In [ ]:
# Discovering outliers with mathematical function

# Z-score
# With Z-score we re-scale and center the data and look for data points which are too far from zero. Data points which are too far from zero will be treated as outliers.
# In the most cases a threshold of 3 or -3 is used. Z-score values greater than or less than 3 or -3 respectively is an outlier.

# Function to compute z-score 
z = np.abs(stats.zscore(indicator_income_data_cleaned[['fertility_rate','urban_population','access_electricity', 'agriculture_forestry_fishing_sector','external_balance_on_goods_and_services','natural_resources_rent','inflation','life_expectancy']]))
print(z)
print()

# defining threshold
threshold = 3

# The first array contains the list of row numbers and the second array respective column numbers
print(np.where(z > 3))
# z[48][6] has a z-score higher than 3.
print(z[48][6])
print(z.shape)


In [ ]:
# exploring outliers

print(indicator_income_data_cleaned.iloc[49,:])

ab = indicator_income_data_cleaned.iloc[indicator_income_data_cleaned.index.get_level_values('country_name') == 'Belarus']
print(ab['inflation'])

# Filter the outliers using Z-score
#indicator_income_data_out = indicator_income_data_cleaned['fertility_rate','urban_population','access_electricity', 'agriculture_forestry_fishing_sector','external_balance_on_goods_and_services','natural_resources_rent','inflation','life_expectancy'](z > 3)]
#print(indicator_income_data_out)

## Explore the data

In [ ]:
sb.heatmap(indicator_income_data_cleaned.isna(), cbar = False);

In [ ]:
sb.countplot(indicator_income_data_cleaned['income_level_label'])

In [ ]:
def plot_scatter_matrix(input_data):
  # Scatterplot Matrix
  sm = pd.plotting.scatter_matrix(input_data[['income_level_label','gni_per_capita_us_dollar','fertility_rate','urban_population','access_electricity', 'agriculture_forestry_fishing_sector','external_balance_on_goods_and_services','natural_resources_rent','inflation','life_expectancy']], figsize=(18, 18), diagonal='hist')
  #Change label rotation
  [s.xaxis.label.set_rotation(90) for s in sm.reshape(-1)]
  [s.yaxis.label.set_rotation(0) for s in sm.reshape(-1)]
  #May need to offset label when rotating to prevent overlap of figure
  [s.get_yaxis().set_label_coords(-0.6,0.5) for s in sm.reshape(-1)]
  #Hide all ticks
  [s.set_xticks(()) for s in sm.reshape(-1)]
  [s.set_yticks(()) for s in sm.reshape(-1)]
  plt.show()

plot_scatter_matrix(indicator_income_data_cleaned)

In [ ]:
def plot_gni_scatter_plot(scatter_data):

  #plot scatter plot with colorized classification and logarithmic scale for gni per capita

  #create income level number
  # low_income = 1
  # lower_middle_income = 2
  # upper_middle_income = 3
  # high_income = 4

  scatter_data['income_classification_number'] = np.nan
  scatter_data.loc[scatter_data['income_level_label'] == 'low_income', 'income_classification_number'] = 1
  scatter_data.loc[scatter_data['income_level_label'] == 'lower_middle_income',['income_classification_number']] = 2
  scatter_data.loc[scatter_data['income_level_label'] == 'upper_middle_income',['income_classification_number']] = 3
  scatter_data.loc[scatter_data['income_level_label'] == 'high_income',['income_classification_number']] = 4


  fig = plt.figure()
  ax = plt.gca()
  plt.scatter(scatter_data['fertility_rate'], scatter_data['gni_per_capita_us_dollar'],\
              c = scatter_data['income_classification_number'],s = 1)
  ax.set_yscale('log')
  ax.set_xlabel('fertility_rate')
  ax.set_ylabel('GNI per Capita in US Dollar')
  plt.show()

  fig = plt.figure()
  ax = plt.gca()
  plt.scatter(scatter_data['urban_population'],scatter_data['gni_per_capita_us_dollar'],\
              c=scatter_data['income_classification_number'],s = 1)
  ax.set_yscale('log')
  ax.set_xlabel('urban_population')
  ax.set_ylabel('GNI per Capita in US Dollar')
  plt.show()

  fig = plt.figure()
  ax = plt.gca()
  plt.scatter(scatter_data['access_electricity'],scatter_data['gni_per_capita_us_dollar'],\
              c=scatter_data['income_classification_number'],s = 1)
  ax.set_yscale('log')
  ax.set_xlabel('access_electricity')
  ax.set_ylabel('GNI per Capita in US Dollar')
  plt.show()

  fig = plt.figure()
  ax = plt.gca()
  plt.scatter(scatter_data['agriculture_forestry_fishing_sector'],scatter_data['gni_per_capita_us_dollar'],\
              c=scatter_data['income_classification_number'],s = 1)
  ax.set_yscale('log')
  ax.set_xlabel('agriculture_forestry_fishing_sector')
  ax.set_ylabel('GNI per Capita in US Dollar')
  plt.show()

  fig = plt.figure()
  ax = plt.gca()
  plt.scatter(scatter_data['inflation'],scatter_data['gni_per_capita_us_dollar'],\
              c=scatter_data['income_classification_number'],s = 1)
  ax.set_yscale('log')
  ax.set_xlabel('inflation')
  ax.set_ylabel('GNI per Capita in US Dollar')
  plt.show()

  fig = plt.figure()
  ax = plt.gca()
  plt.scatter(scatter_data['external_balance_on_goods_and_services'],scatter_data['gni_per_capita_us_dollar'],\
              c=scatter_data['income_classification_number'].values,s = 1)
  ax.set_yscale('log')
  ax.set_xlabel('external_balance_on_goods_and_services')
  ax.set_ylabel('GNI per Capita in US Dollar')
  plt.show()

  fig = plt.figure()
  ax = plt.gca()
  plt.scatter(scatter_data['natural_resources_rent'],scatter_data['gni_per_capita_us_dollar'],\
              c=scatter_data['income_classification_number'],s = 1)
  ax.set_yscale('log')
  ax.set_xlabel('natural_resources_rent')
  ax.set_ylabel('GNI per Capita in US Dollar')
  plt.show()

  fig = plt.figure()
  ax = plt.gca()
  plt.scatter(scatter_data['life_expectancy'],scatter_data['gni_per_capita_us_dollar'],\
              c=scatter_data['income_classification_number'],s = 1)
  ax.set_yscale('log')
  ax.set_xlabel('life_expectancy')
  ax.set_ylabel('GNI per Capita in US Dollar')
  plt.show()


plot_gni_scatter_plot(indicator_income_data_cleaned)



In [ ]:
def plot_correlation_matrix(input_data):
  # Full correlation matrix

  colum_names = ['income_level_label','gni_per_capita_us_dollar','fertility_rate','urban_population','access_electricity', 'agriculture_forestry_fishing_sector','external_balance_on_goods_and_services','natural_resources_rent','inflation','life_expectancy']
  # Correlation matrix
  correlations = input_data[['income_level_label','gni_per_capita_us_dollar','fertility_rate','urban_population','access_electricity', 'agriculture_forestry_fishing_sector','external_balance_on_goods_and_services','natural_resources_rent','inflation','life_expectancy']].corr()
  # Plot figsize
  fig, ax = plt.subplots(figsize=(10, 10))
  # Generate Color Map
  colormap = sb.diverging_palette(220, 10, as_cmap=True)
  # Generate Heat Map, allow annotations and place floats in map
  sb.heatmap(correlations, cmap=colormap, annot=True, fmt=".2f")
  ax.set_xticklabels(
      colum_names,
      rotation=45,
      horizontalalignment='right'
  );
  ax.set_yticklabels(colum_names);
  plt.show()

plot_correlation_matrix(indicator_income_data_cleaned)  

# ML Algorithmus

In [ ]:
from sklearn import model_selection
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib

In [ ]:
# 20 selected countries( selection s.t. Training/Test set (20 countries) 5 countries per class 1,2,3,4 ... Gleiche Verteilung) 
twenty_countries = ['USA','CHL','AUT','BHR','ISR','TUR','COL','CHN','BRA','AZE','NGA','IND','BOL','CIV','EGY','AFG','BEN','TCD','BFA','CMR']  
parameters = ['fertility_rate', 'urban_population', 'access_electricity', 'agriculture_forestry_fishing_sector','external_balance_on_goods_and_services','life_expectancy','income_level_label']

indicator_income_data_cleaned.reset_index(inplace = True)
indicator_income_data_cleaned.set_index(['calendar_year','country_name'], inplace = True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

indicator_income_data_cleaned['iso_alpha3_code'] = indicator_income_data_cleaned['iso_alpha3_code'].astype('string')

#indicator_income_data_sel = indicator_income_data_cleaned[indicator_income_data_cleaned['iso_alpha3_code'].isin(twenty_countries)]
indicator_income_data_sel = indicator_income_data_cleaned

#leave only parameters (colums) of interest:
indicator_income_data_sel = indicator_income_data_sel[parameters]

indicator_income_data_sel_2014 = indicator_income_data_sel.loc[2014]
indicator_income_data_sel_2015 = indicator_income_data_sel.loc[2015]
indicator_income_data_sel_2016 = indicator_income_data_sel.loc[2016]
indicator_income_data_sel_2017 = indicator_income_data_sel.loc[2017]
indicator_income_data_sel_2018 = indicator_income_data_sel.loc[2018]

indicator_income_data_sel_2014.head(25)
#indicator_income_data

In [ ]:
# EXPLORE 

# Histogram of the class distribution
#print (min(y),max(y))
#y.hist(bins=6);

In [ ]:
# Histogram of the feature distributions
#X.hist(bins=10,figsize=(10,10));

In [ ]:
# Print the correlation coefficient of each feature with the class
corr_matrix = indicator_income_data_sel_2014.corr()
#print(corr_matrix['income_classification_number'].sort_values())

## So, population_total is not a factor in the income level. Can be left out as parameter.

In [ ]:
# Full correlation matrix

colum_names = parameters
# Correlation matrix
correlations = indicator_income_data_sel_2014.corr()
# Plot figsize
fig, ax = plt.subplots(figsize=(10, 10))
# Generate Color Map
colormap = sns.diverging_palette(220, 10, as_cmap=True)
# Generate Heat Map, allow annotations and place floats in map
sns.heatmap(correlations, cmap=colormap, annot=True, fmt=".2f")
ax.set_xticklabels(
    colum_names,
    rotation=45,
    horizontalalignment='right'
);
ax.set_yticklabels(colum_names);
#plt.show()

In [ ]:
# Scatterplot Matrix
sm = pd.plotting.scatter_matrix(indicator_income_data_sel_2014, figsize=(18, 18), diagonal='hist')
#Change label rotation
[s.xaxis.label.set_rotation(40) for s in sm.reshape(-1)]
[s.yaxis.label.set_rotation(0) for s in sm.reshape(-1)]
#May need to offset label when rotating to prevent overlap of figure
[s.get_yaxis().set_label_coords(-0.6,0.5) for s in sm.reshape(-1)]
#Hide all ticks
[s.set_xticks(()) for s in sm.reshape(-1)]
[s.set_yticks(()) for s in sm.reshape(-1)]
#plt.show()

In [ ]:
  # set random seed in order to ensure reproducible results
  np.random.seed(20210112)
  rnd = np.random.randint(0,100)

In [ ]:
def separate_training_test_data(input_data, rnd):

  # retrieve number of columns of input_data
  n_cols = len(input_data.columns)
  
  # split up input_data into features and targets
  features = input_data.reset_index().iloc[:, 1:n_cols].values
  targets = input_data.reset_index().iloc[:, n_cols:(n_cols+1)].values.reshape(-1)

  # separate training and test data with test_size = 0.2
  features_train, features_test, targets_train, targets_test = train_test_split(features, targets, test_size = 0.2, random_state = rnd)

  return features_train, features_test, targets_train, targets_test

features_train, features_test, targets_train, targets_test = separate_training_test_data(indicator_income_data_sel_2014, rnd)

In [ ]:
def perform_random_forest_classification(features, targets, rnd):

  # build classification model based on random forest
  rfcl = RandomForestClassifier(n_estimators = 50, random_state = rnd) 

  # set k and initialize cross-validation settings
  k = 10
  cv = KFold(n_splits = k, random_state = rnd, shuffle = True)

  scores = []

  # perform train-/validation-split
  for train_index, validation_index in cv.split(features):
    
    features_train, features_validation = features[train_index], features[validation_index]
    targets_train, targets_validation = targets[train_index], targets[validation_index]
    
    # fit to random forest classification model
    rfcl.fit(features_train, targets_train)
    
    # compute score that compares predictions of model with true values of 
    scores.append(rfcl.score(features_validation, targets_validation))

  print('mean accuracy (train-validate): ', mean(scores))

  return rfcl

random_forest_classifier = perform_random_forest_classification(features_train, targets_train, rnd)
print('mean accuracy (test): ', random_forest_classifier.score(features_test, targets_test))

In [ ]:
# SPLIT DATA FOR TRAINING AND TESTING

# Separate dataset as feature variables (unsere Parameter) and response variable (income_level_label)
X = indicator_income_data_sel_2014[parameters].values
y = indicator_income_data_sel_2014.income_level_label.values

# Split into Traindata and Testdata
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=50) #80/20 split
#y

#bins = (1,2,3,4,5) # # of bins muss # labels(klassen) + 1 sein
#labels = ['low','med_low','med_high','high']
#y_train = pd.cut(y_train, bins = bins, labels = labels)
#y_test = pd.cut(y_test, bins = bins, labels = labels)

#sns.countplot(y_test)
#plt.show()

#print(y_train)
#print(y_test)

#create and fit the model
rfcl = RandomForestClassifier(n_estimators=50) 

rfcl.fit(X_train, y_train)
rfcl.score(X_test,y_test)

from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
#kf

for train_index, test_index in kf.split(X):
  print(train_index, test_index)

In [ ]:
# measure performance

def get_score(model, X_train, X_test, y_train, y_test):
  model.fit(X_train, y_train)
  return model.score(X_test, y_test)

get_score(RandomForestClassifier(), X_train, X_test, y_train, y_test)

In [ ]:
from sklearn.model_selection import StratifiedKFold #dasselbe wie KFold, nur hier werden Kategorien(low,med,..) Uniform verteilt.

folds = StratifiedKFold(n_splits=3)

scores_rf = []

for train_index, test_index in folds.split(X,y):
  X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
  scores_rf.append(get_score(RandomForestClassifier(100), X_train, X_test, y_train, y_test))

In [ ]:
scores_rf

In [ ]:
from sklearn.model_selection import cross_val_score #cross_val_score verwendet stratified kfold

cross_val_score(RandomForestClassifier(n_estimators=50),X, y, cv=3)

In [ ]:
# Parameter tuning mit kfold cross validation

scores1 = cross_val_score(RandomForestClassifier(n_estimators=5),X, y, cv=10)
np.average(scores1)


In [ ]:
scores2 = cross_val_score(RandomForestClassifier(n_estimators=20),X, y, cv=10)
np.average(scores2)

In [ ]:
scores3 = cross_val_score(RandomForestClassifier(n_estimators=30),X, y, cv=10)
np.average(scores3)

In [ ]:
scores4 = cross_val_score(RandomForestClassifier(n_estimators=40),X, y, cv=10)
np.average(scores4)